In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [0]:
dbutils.fs.ls("/FileStore/tables")

Out[5]: [FileInfo(path='dbfs:/FileStore/tables/0ed442ca38ad_user_table/', name='0ed442ca38ad_user_table/', size=0, modificationTime=1704122067957),
 FileInfo(path='dbfs:/FileStore/tables/authentication.csv/', name='authentication.csv/', size=0, modificationTime=1704122067957),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials-1.csv', name='authentication_credentials-1.csv', size=97, modificationTime=1702911405000),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials-2.csv', name='authentication_credentials-2.csv', size=97, modificationTime=1702911558000),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials-3.csv', name='authentication_credentials-3.csv', size=87, modificationTime=1703678451000),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv/', name='authentication_credentials.csv/', size=0, modificationTime=1704122067957),
 FileInfo(path='dbfs:/FileStore/tables/csv_files/', name='csv_files/', size=0, modificationTime=1704122067957),
 FileInfo(path='dbfs:/FileStore/tables/date_details.json', name='date_details.json', size=284166, modificationTime=1703945051000),
 FileInfo(path='dbfs:/FileStore/tables/downloaded_file.json', name='downloaded_file.json', size=48, modificationTime=1703945574000),
 FileInfo(path='dbfs:/FileStore/tables/downloaded_file2.json', name='downloaded_file2.json', size=60, modificationTime=1703954387000),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1704122067957),
 FileInfo(path='dbfs:/FileStore/tables/json_files/', name='json_files/', size=0, modificationTime=1704122067957),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1704122067957),
 FileInfo(path='dbfs:/FileStore/tables/single_json_file.json', name='single_json_file.json', size=293, modificationTime=1701857903000),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1704122067957)]

In [0]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0e0816526d11-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0e0816526d11-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[8]: True

In [0]:
# list the topics stored on the mounted S3 bucket
display(dbutils.fs.ls("/mnt/user-0e0816526d11-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.geo/,0e0816526d11.geo/,0,1704122522372
dbfs:/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.pin/,0e0816526d11.pin/,0,1704122522372
dbfs:/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.user/,0e0816526d11.user/,0,1704122522372


In [0]:
# create path to topic files
file_location = "/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.pin/partition=0/*.json"
# specify file type
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# load JSONs from mounted S3 bucket to Spark dataframe
df_pin = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)

# cleaning Pinterest dataframe
df_pin = df_pin.replace("No description available Story format", None)
df_pin = df_pin.replace("null", None)
df_pin = df_pin.replace("User Info Error", None)
df_pin = df_pin.replace("Image src error", None)
df_pin = df_pin.replace("N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e", None)
df_pin = df_pin.replace("No Title Data Available", None)

# transforms the 'follower_count' col from string to integer and checks if the value matches a pattern that contains either k or M 
df_pin = df_pin.withColumn("follower_count", when(
    col("follower_count").rlike("\d+k"),(regexp_extract(col("follower_count"),"(\d+)",1).cast("integer") * 1000)).when(col("follower_count").rlike("\d+M"),(regexp_extract(col("follower_count"), "(\d+)", 1).cast("integer") * 1000000))
# otherwise, if it doesn't matches it leaves the full integer value
.otherwise(col("follower_count").cast("integer")))

# cleaning the 'save_location' column by removing 'Local save in ' text and just leaving the path for the 'save_location' column
df_pin = df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

# Renaming the column
df_pin = df_pin.withColumnRenamed("index", "ind")

df_pin = df_pin.dropDuplicates(["unique_id"])
df_pin = df_pin.na.drop()

# rearranging the Pinterest columns
reorder_col = ["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category"]
df_pin = df_pin.select(reorder_col)

# Show the table of the Pinterest data
display(df_pin)

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
8578,036ec267-658b-4dad-a097-8ef0b4f30c0d,Snake Collarbone Temporary Tattoo. Halloween Tattoos Spooky | Etsy,"Snake temporary tattoo for collarbone. Spooky 3D tattoo for Halloween Includes 2 copies, 1 for the right side and 1 for the left side Apply with water and remove anytime with ba…",0,Livitrevisani,"Finger Tattoo Designs,Small Tattoo Designs,Finger Tattoos,Small 3d Tattoos,Cool Tattoos,Line Work Tattoo,Back Tattoo,Collarbone Tattoo,Tattoo Neck",image,https://i.pinimg.com/originals/d8/b7/a1/d8b7a11a3fe64e7fab91d3afb1536537.jpg,/data/tattoos,tattoos
2482,08604f20-fa17-4b9a-9949-781717eca6cd,FORNT PORCH CHRISTMAS DECORATING IDEAS,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",46000,"Life on Summerhill | Home, Holiday Decor & DIY Website","Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",video,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,/data/christmas,christmas
1545,0c629541-cc5b-4b61-bd3f-613346893258,Oil Cleansing Method: What it is and why you should do it.,"The oil cleansing method is a simple, natural way to have clean skin without using toxic chemicals. It will leave your face radiant and soft!",85000,Thank Your Body,"Beauty Care,Diy Beauty,Beauty Hacks,Beauty Shop,Oil Cleansing Method,Goji,Korean Skincare Routine,Homemade Beauty Products,Organic Beauty",image,https://i.pinimg.com/originals/2c/1c/da/2c1cda7da86ee711536bbacfe89c75c9.jpg,/data/beauty,beauty
10552,17301b8c-ccbb-42ad-b2df-3876df35b1cd,The Prowler Emergency Response Vehicle,"January 23, 2009 We can see emergency responders queueing up for this one. ATVCorp has created versions of its Prowler Light Tactical All Terrain Vehicle (LTATV) for fire-fighti…",314,p hump,"Army Vehicles,Armored Vehicles,Go Kart Buggy,Atv Attachments,Aviation Fuel,Special Forces,Special Ops,Terrain Vehicle,Emergency Response",image,https://i.pinimg.com/originals/14/6b/6b/146b6b1a1127773f7046e08550e68ed0.jpg,/data/vehicles,vehicles
46,19234073-8905-4885-b0d5-98e0b84cbf27,10 Watercolor Hacks For Beginners | Tips and Tricks to Making Watercolor Painting Easier,Mountain monologue watercolor,27000,"It's me, JD | DIY, Crafts, Home & Organization","Arte Inspo,Kunst Inspo,Watercolor Artists,Watercolor Ideas,Simple Watercolor,Tattoo Watercolor,Watercolor Techniques,Watercolor Animals,Watercolor Illustration",image,https://i.pinimg.com/originals/fd/54/89/fd548935dcb13545120a2115baaa41d9.jpg,/data/art,art
9672,1a5050a4-ac82-472c-aeff-3457de8bbee9,"Lake Moraine, Banff, Canada","On our Calgary to Vancouver road trip we got to spend time in Banff, Canada. We visited the stunning Lake Moraine as well as Lake Louise and plenty of other of Canada's natural…",7000,Travel Mad Mum,"Beautiful Places To Travel,Cool Places To Visit,Beautiful Roads,Beautiful Scenery,Wonderful Places,Lac Moraine,Moraine Lake,Lac Louise,Natur Wallpaper",video,https://i.pinimg.com/videos/thumbnails/originals/57/9d/52/579d52242ae5609978c754d393e44e81.0000001.jpg,/data/travel,travel
5730,1e1f0c8b-9fcf-460b-9154-c775827206eb,Island Oasis Coupon Organizer,"Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the ""basic"" couponer - holds up to 500 coupons with ease, and is made long enough so that you…",0,Consuelo Aguirre,"Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons",image,https://i.pinimg.com/originals/65/bb/ea/65bbeaf458907bb079317d8303c4fa0e.jpg,/data/finance,finance
1555,1f20551b-a21c-4ff0-b20d-49e95fdc50d3,Skin P

In [0]:
file_location = "/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.geo/partition=0/*.json"
# specify file type
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# load JSONs from mounted S3 bucket to Spark dataframe
df_geo = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)

# created a new column containing latitude and longitude
df_geo = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

# dropping columns
df_geo = df_geo.drop("latitude", "longitude")
df_geo = df_geo.withColumn("timestamp", col("timestamp").cast("timestamp"))
df_geo = df_geo.dropDuplicates(["ind","country", "coordinates", "timestamp"])
# df_geo = df_geo.na.drop()

geo_reorder_col = ["ind", "country", "coordinates", "timestamp"]
df_geo = df_geo.select(geo_reorder_col)

df_geo = df_geo.sort("country")

# Show the table of the Geolocation data
display(df_geo)

ind,country,coordinates,timestamp
3599,Afghanistan,"List(-88.5478, -174.971)",2019-03-03T06:13:41.000+0000
46,Afghanistan,"List(4.21689, -145.82)",2018-09-15T09:16:57.000+0000
7528,Albania,"List(-89.9787, -173.293)",2020-08-28T03:52:47.000+0000
808,Albania,"List(-71.6856, -179.126)",2019-01-03T15:43:12.000+0000
3089,Albania,"List(-89.9787, -173.293)",2018-02-28T05:31:29.000+0000
2411,Albania,"List(-71.6856, -179.126)",2020-11-15T17:10:07.000+0000
6398,Albania,"List(-3.29267, -179.726)",2020-01-19T01:49:05.000+0000
1555,Algeria,"List(-1.7046, -55.332)",2017-12-24T18:41:23.000+0000
7768,Algeria,"List(-88.355, -96.0131)",2018-05-25T21:34:19.000+0000
9590,Algeria,"List(-64.3876, -174.281)",2019-05-20T19:48:48.000+0000


In [0]:
file_location = "/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.user/partition=0/*.json"
# specify file type
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# load JSONs from mounted S3 bucket to Spark dataframe
df_user = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)

df_user = df_user.withColumn("user_name", concat(col("first_name"),lit(" "),col("last_name")))
df_user = df_user.drop("first_name","last_name","index")
df_user = df_user.withColumn("date_joined", col("date_joined").cast("timestamp"))
df_user = df_user.dropDuplicates(["user_name", "age", "date_joined"])
df_user = df_user.na.drop()

user_reorder_col = ["ind","user_name","age","date_joined"]
df_user = df_user.select(user_reorder_col)

df_user = df_user.sort("user_name")

# Show the table of the User data
display(df_user)

ind,user_name,age,date_joined
1422,Aaron Abbott,20,2015-10-31T18:51:07.000+0000
2411,Aaron Bartlett,21,2015-11-24T02:15:36.000+0000
3419,Aaron Jenkins,47,2017-04-26T03:26:23.000+0000
7528,Abigail Ali,20,2015-10-24T11:23:51.000+0000
3800,Adam Armstrong,20,2015-12-17T08:43:40.000+0000
831,Aimee Collins,28,2016-06-16T02:06:38.000+0000
9546,Alex Barnes,20,2016-02-27T21:13:44.000+0000
2293,Alex Bean,24,2016-01-11T23:08:27.000+0000
6566,Alexander Perez,31,2017-08-04T14:30:22.000+0000
3599,Alexandria Alvarado,20,2015-10-23T04:13:23.000+0000


In [0]:
dbutils.fs.unmount("/mnt/user-0e0816526d11-bucket")

/mnt/user-0e0816526d11-bucket has been unmounted.
Out[2]: True

In [0]:
# Finding the most popular Pinterest category in each country.

df_pin_geo = df_pin.join(df_geo, "ind").groupBy(df_geo.country, df_pin.category).agg(count("category").alias("category_count")).orderBy(desc("category_count"))

display(df_pin_geo)


country,category,category_count
Australia,mens-fashion,2
Austria,travel,2
Armenia,diy-and-crafts,2
India,travel,1
French Guiana,quotes,1
Cocos (Keeling) Islands,vehicles,1
Cambodia,diy-and-crafts,1
Grenada,vehicles,1
Armenia,christmas,1
Central African Republic,christmas,1


In [0]:
# Finding the most popular category each year

df_post_year = df_pin.join(df_geo, "ind").groupBy(year("timestamp").alias("post_year"), "category").agg(count("category").alias("category_count"))

post_year_filter = df_post_year.filter((col("post_year") >= 2018) & (col("post_year") <= 2022))

popular_cat_each_year = post_year_filter.groupBy("post_year").agg(first("category").alias("category"), first("category_count").alias("category_count")).orderBy(desc("post_year"))

display(popular_cat_each_year)

post_year,category,category_count
2022,beauty,2
2021,event-planning,1
2020,diy-and-crafts,1
2019,christmas,2
2018,mens-fashion,2


In [0]:
# Finding user with most follower in each country

df_c_pn_fc = df_pin.join(df_geo, "ind").groupBy(df_geo.country, df_pin.poster_name).agg(first(df_pin.follower_count).alias("follower_count")).orderBy(desc("follower_count"))
display(df_c_pn_fc)

# country with most user followers

df_c_fc = df_c_pn_fc.orderBy(desc("follower_count"))
df_c_fc = df_c_fc.drop("poster_name")
df_c_fc.show(1)

country,poster_name,follower_count
Azerbaijan,Style Me Pretty,6000000
American Samoa,BuzzFeed,5000000
Christmas Island,Instructables,3000000
Bangladesh,Architectural Digest,3000000
Afghanistan,Walmart,2000000
Andorra,Teachers Pay Teachers,1000000
Australia,Cultura Colectiva,1000000
Armenia,Michelle {CraftyMorning.com},892000
Central African Republic,PureWow,868000
Argentina,Next Luxury,800000


+----------+--------------+
 country|follower_count|
+----------+--------------+
Azerbaijan| 6000000|
+----------+--------------+
only showing top 1 row

In [0]:
# Finding the most popular category for different age groups

# age groups:
# 18 <-> 24
# 25 <-> 35
# 36 <-> 50
# +50

df_age_group = df_user.withColumn("age_group", when((df_user.age >=18) & (df_user.age <= 24), "18 - 24").when((df_user.age >=25) & (df_user.age <= 35), "25 - 35").when((df_user.age >=36) & (df_user.age <= 50), "36 - 50").otherwise("Invalid"))

df_pin_age_group_join = df_pin.join(df_age_group, "ind").groupBy(df_age_group.age_group, df_pin.category).agg(count("*").alias("category_count")).orderBy(desc("category_count"))

popular_cat_by_age_group = df_pin_age_group_join.groupBy("age_group").agg(first("category").alias("category"), first("category_count").alias("category_count"))

display(popular_cat_by_age_group)

age_group,category,category_count
18 - 24,art,5
25 - 35,travel,4
36 - 50,diy-and-crafts,3
Invalid,education,2


In [0]:
# Finding the median follower count for different age groups

df_median = df_pin.join(df_age_group, "ind").groupBy(df_age_group.age_group, df_pin.follower_count).agg(count("*").alias("median_follower_count"))

df_median_age_group = df_median.groupBy("age_group").agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count")).orderBy(asc("age_group"))

display(df_median_age_group)


age_group,median_follower_count
18 - 24,89000
25 - 35,42000
36 - 50,6000
Invalid,5000


In [0]:
# Finding how many users have joined each year?

df_user_post_year = df_user.withColumn("post_year", year("date_joined"))

df_user_joined = df_user_post_year.groupBy("post_year").agg(count("*").alias("number_users_joined"))

user_post_year_filter = df_user_joined.filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

user_joined_each_year = user_post_year_filter.groupBy("post_year").agg(first("number_users_joined").alias("number_users_joined")).orderBy(desc("post_year"))

display(user_joined_each_year)

post_year,number_users_joined
2017,11
2016,37
2015,30


In [0]:
# Finding the median follower count of users that have joined between 2015 to 2020

df_pin_user_median_follower_count = df_pin.join(df_user_post_year, "ind").groupBy(df_user_post_year.post_year, df_pin.follower_count).agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

df_user_post_year_filter = df_pin_user_median_follower_count.filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

user_mfc = df_user_post_year_filter.groupBy("post_year").agg(first("median_follower_count").alias("median_follower_count")).orderBy(desc("post_year"))

display(user_mfc)

post_year,median_follower_count
2017,112
2016,25000
2015,42000


In [0]:
# Finding the median follower count of users based on their joining and age group

df_pin_age_group_join2 = df_pin.join(df_age_group, "ind").groupBy(df_age_group.age_group, df_pin.follower_count).agg(count("*").alias("median_follower_count")).orderBy(desc("median_follower_count"))

df_pin_user_median_follower_count = df_pin.join(df_user_post_year, "ind").groupBy(df_user_post_year.post_year, df_pin.follower_count).agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

df_age_group_post_year2 = df_pin_age_group_join2.join(df_pin_user_median_follower_count, "follower_count").groupBy(df_pin_age_group_join2.age_group, df_pin_user_median_follower_count.post_year).agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count")).orderBy(asc("age_group"))

# filters the post year between 2015 to 2020
df_age_group_post_year_filter = df_age_group_post_year2.filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

display(df_age_group_post_year_filter)

age_group,post_year,median_follower_count
18 - 24,2015,110000
18 - 24,2016,25000
18 - 24,2017,5000
25 - 35,2015,42000
25 - 35,2016,22000
25 - 35,2017,8000
36 - 50,2017,314
36 - 50,2015,25000
36 - 50,2016,7000
Invalid,2017,5000
